In [1]:
# import libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re

#### 1)   create a name list for searching

In [2]:
# read in the csv file for search
df=pd.read_csv("name_4.csv",index_col=False, encoding='latin-1')['Nombre']    # specify the file name
print(df)

search_list=df.tolist()

0             CARLOS IVAN VALENZUELA BENITEZ
1               CARLOS IVAN VARGAS HERNANDEZ
2       CARLOS IVAN VLADIMIR ALVAREZ SAUCEDO
3                   CARLOS IVAN ZEA OLIVARES
4           CARLOS JACINTO CAMACHO MANDUJANO
                        ...                 
3602           ZAIRA BERENICE GOMEZ MARTINEZ
3603           ZEFERINO CARLOS RAMOS AGUILAR
3604              ZEIDY BRENDA LOPEZ ESTRADA
3605         ZUEMY MARIANA SALINAS HERNANDEZ
3606           ZULLY BERENICE MARTINEZ LOPEZ
Name: Nombre, Length: 3607, dtype: object


**NOTE**: special characters/dots, etc., in the employee names should be removed before using for query.

#### 2) define a function for user name/ID extraction

In [3]:
# add 'chromedriver' executable to PATH
import os
os.environ['PATH']+=";C:\Program Files (x86)\Google\Chrome\Application"  

In [4]:
# specify the target webpage for scraping
start_url="https://servidorespublicos.gob.mx/"   

# initiate the driver
chrome_options = webdriver.chrome.options.Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

try:
    driver.implicitly_wait(15)
    driver.get(start_url)
except Exception as e:
    print(e)
    
# click "De acuerdo"   
while True:
    try:
        driver.find_element_by_xpath("//*[@id='modalInfo']/div/div/div[3]/button").click()
        break
    except: 
        continue

In [5]:
# define the function to send query and extract user information
def search_and_extraction(search_name):
    # send queries
    iptbox=driver.find_element_by_css_selector('input[name="nombre"]')
    btn=driver.find_element_by_css_selector('button[value="Buscar"]')
    
    iptbox.clear()
    iptbox.send_keys(search_name)
    btn.submit()
    # create a list to save the returned list of employees
    time.sleep(6)
    returned_list=driver.find_elements_by_xpath("//*[@id='tableBusqueda']/tbody/tr/td/a")
    # extract user name & ID (three candidate user identifiers: 'idUsrDecnet','CUSP','RFC')
    space=' '
    results=[]
    for name in returned_list:
        # get 'href' and its attributes
        href=name.get_attribute('href')
        href_elements=name.get_attribute('href').split(';')
        # get user identifiers using regular expressions
        if len(href_elements)==9:
            userName=name.text
            userID_1=re.findall('[0-9]+', href_elements[5])[0]             # 'idUsrDecnet'
            userID_2=' '                                                   # 'CUSP' is missing
            userID_3=re.findall('[0-9A-Za-z]+', href_elements[6])[1]       # 'RFC' 
            results.append((userName,userID_1, userID_2, userID_3, href))
        
        elif len(href_elements)==10:
            userName=name.text
            userID_1=re.findall('[0-9]+', href_elements[5])[0]             # 'idUsrDecnet'
            userID_2=re.findall('[0-9A-Za-z]+', href_elements[6])[1]       # 'CUSP'  
            userID_3=re.findall('[0-9A-Za-z]+', href_elements[7])[1]       # 'RFC'
            results.append((userName,userID_1, userID_2, userID_3, href))
    
    return results

#### 3) send names for searching and retrieve user IDs

In [ ]:
# call the function
search_results=[]
for name in search_list:
    search_results += search_and_extraction(name)   

In [7]:
# save the results as dataframe
df=pd.DataFrame(search_results)
df.columns=['Nombre','idUsrDecnet','CUSP', 'RFC', 'href']
print(df)

# export the results as csv file
df.to_csv('Outcomes/2. name_info/Name_Info_103.csv', encoding='utf-8-sig', index=False)

                            Nombre idUsrDecnet                CUSP  \
0            SONIA VENTURA HILERIO      520665                       
1            SONIA VENTURA HILERIO      520665                       
2            SONIA VENTURA HILERIO      520665                       
3            SONIA VENTURA HILERIO      520665                       
4            SONIA VENTURA HILERIO      520665                       
...                            ...         ...                 ...   
2817  TANIA LIZBETH GONZALEZ ROJAS     1009802                       
2818            TANIA ROMAN GUZMAN      428188  ROGT691110MDFMZN01   
2819            TANIA ROMAN GUZMAN      428188                       
2820            TANIA ROMAN GUZMAN      428188                       
2821          TANIA ROMERO SEVILLA     3529872  ROST911112MDFMVN03   

                RFC                                               href  
0     VEHS6907082F1  https://servidorespublicos.gob.mx/declaracione...  
1     VEHS690

**NOTE:** The search process might break down if the target element is not found. In this case, we simply re-start scraping from the last name being processed. 

In [8]:
print(name)

TANIA ROSALINDA MÉNDEZ LÓPEZ
